In [ ]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2
import vpype
from skimage import io
from pathlib import Path

import bezier

from sklearn.preprocessing import minmax_scale
from skimage import feature
from genpen.utils import Paper

In [ ]:
# make page
paper_size = '11x14 inches'
border:float=20
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
lines = []

node_sets = []
n_lines = 1200
n_nodes_per_line = 20
y_start = 1
y_end = 10
x_start = 1
x_end = 10
node_x_centers = np.linspace(x_start, x_end, n_lines)
std_scale = 0.02
n_eval_points = 250

### initialize vals
node_ys = np.linspace(y_start, y_end, n_nodes_per_line)
centered_node_xs = np.zeros(node_ys.shape)
bez_eval_end_center = 1
bez_eval_end_noise = 0
bez_eval_end_limit = 1.1
bez_eval_end_std_scale = 0.01

for i, node_x_center in enumerate(node_x_centers):
#     node_ys = np.flipud(node_ys)
#     centered_node_xs = np.flipud(centered_node_xs)
    new_x_noise = np.random.randn(n_nodes_per_line) * std_scale
#     bez_eval_end_noise = bez_eval_end_noise + np.random.randn() * bez_eval_end_std_scale
    centered_node_xs = centered_node_xs + new_x_noise
    node_xs = node_x_center + centered_node_xs
    node_xs[0:3] = node_x_center
#     node_xs[-1] = node_x_center
    nodes = np.asfortranarray([
        node_xs,
        node_ys,
        ])
    curve = bezier.Curve(nodes, degree=(nodes.shape[1]-1))
#     bez_eval_end = bez_eval_end_center + bez_eval_end_noise
#     if bez_eval_end > bez_eval_end_limit:
#         bez_eval_end = 1 + np.random.randn()*0.02
#     eval_points = np.linspace(0, bez_eval_end, n_eval_points)
    eval_points = np.linspace(0, 1, n_eval_points)
    x, y = curve.evaluate_multi(eval_points)
    if i % 2:
        x = np.flipud(x)
        y = np.flipud(y)
    lines.append(np.stack([x, y]).T)
    node_sets.append(np.stack([node_xs, node_ys]).T)

In [ ]:
ls = [LineString(l) for l in lines]

In [ ]:
mls = gp.make_like(gp.merge_LineStrings(ls), drawbox)

In [ ]:
mask = drawbox.centroid.buffer(100)

In [ ]:
in_mask = mls.intersection(mask)

In [ ]:
in_mask = sa.rotate(in_mask, -90)

In [ ]:
split_point = 2100

In [ ]:
layer1 = in_mask[:2100]
layer2 = in_mask[2100:]

In [ ]:
layers = []
layers.append(LineString(np.concatenate([np.array(l) for l in layer1])))
layers.append(LineString(np.concatenate([np.array(l) for l in layer2])))

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

sk.penWidth('0.3')
# sk.vpype(f'linesimplify --tolerance 0.3mm')

sk.display()

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/169_all_curve.svg')

# lets get more spacing, to work better with staedtlers

In [ ]:
# make page
paper_size = '11x14 inches'
border:float=20
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
lines = []

node_sets = []
n_lines = 800
n_nodes_per_line = 20
y_start = 1
y_end = 10
x_start = 1
x_end = 10
node_x_centers = np.linspace(x_start, x_end, n_lines)
std_scale = 0.025
n_eval_points = 250

### initialize vals
node_ys = np.linspace(y_start, y_end, n_nodes_per_line)
centered_node_xs = np.zeros(node_ys.shape)
bez_eval_end_center = 1
bez_eval_end_noise = 0
bez_eval_end_limit = 1.1
bez_eval_end_std_scale = 0.01

for i, node_x_center in enumerate(node_x_centers):
#     node_ys = np.flipud(node_ys)
#     centered_node_xs = np.flipud(centered_node_xs)
    new_x_noise = np.random.randn(n_nodes_per_line) * std_scale
#     bez_eval_end_noise = bez_eval_end_noise + np.random.randn() * bez_eval_end_std_scale
    centered_node_xs = centered_node_xs + new_x_noise
    node_xs = node_x_center + centered_node_xs
    node_xs[0:3] = node_x_center
#     node_xs[-1] = node_x_center
    nodes = np.asfortranarray([
        node_xs,
        node_ys,
        ])
    curve = bezier.Curve(nodes, degree=(nodes.shape[1]-1))
#     bez_eval_end = bez_eval_end_center + bez_eval_end_noise
#     if bez_eval_end > bez_eval_end_limit:
#         bez_eval_end = 1 + np.random.randn()*0.02
#     eval_points = np.linspace(0, bez_eval_end, n_eval_points)
    eval_points = np.linspace(0, 1, n_eval_points)
    x, y = curve.evaluate_multi(eval_points)
    if i % 2:
        x = np.flipud(x)
        y = np.flipud(y)
    lines.append(np.stack([x, y]).T)
    node_sets.append(np.stack([node_xs, node_ys]).T)

In [ ]:
ls = [LineString(l) for l in lines]

mls = gp.make_like(gp.merge_LineStrings(ls), drawbox)

mask = drawbox.centroid.buffer(100)

in_mask = mls.intersection(mask)

in_mask = sa.rotate(in_mask, -90)

In [ ]:
inds = np.arange(len(in_mask))

split_inds = np.array_split(inds[200:], 4)

In [ ]:
in_mask.geoms[0]

In [ ]:

layers = []
for ind in split_inds:
    layer = [in_mask[i] for i in ind]
    layers.append(LineString(np.concatenate([np.array(l) for l in layer])))

sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

sk.penWidth('0.3')
# sk.vpype(f'linesimplify --tolerance 0.3mm')

sk.display()

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/171_all_curve.svg')